In [15]:
import math
import numpy as np
from scipy.stats import binom
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.spatial import KDTree

In [5]:
k_b = 1.38*10**-23
T = 310.15
n_vis = 0.7*10**-3
r_can = 10*10**-6
r_T = 5*10**-6
D_can = (k_b*T)/(6*math.pi*n_vis*r_can)
D_T = (k_b*T)/(6*math.pi*n_vis*r_T)
print(D_can)
print(D_T)

3.2437823679968016e-14
6.487564735993603e-14


In [23]:
# Define parameters
k_b = 1.38e-23
T_kelvin = 310.15
n_vis = 0.7e-3
r_can = 10e-6  
r_T = 5e-6  
n = n_2 = 10000  
grid_size_um = 0.31 * 5120  # 0.31 µm per pixel, 5120x5120 grid
A_grid = (grid_size_um**2) * 1e-12 # converting to m²
t = 6 * 60

# Diffusion coefficients for type 1 (D_1) and type 2 (D_2) cells
D_1 = (k_b * T_kelvin) / (6 * math.pi * n_vis * r_can)
D_2 = (k_b * T_kelvin) / (6 * math.pi * n_vis * r_T)

# Combined diffusion coefficient (relative velocity)
D_rel = D_1 + D_2

# Interaction probability q
q = 1 - np.exp(-(n_2 / A_grid) * np.pi * 2 * D_rel * t)

# Transition probability P(k -> k+m)
def transition_probability(k, m):
    """
    Calculate the transition probability from state k to state k+m
    P(k -> k+m) = binomial(n-k, m) * q^m * (1 - q)^(n-k-m)
    """
    return binom.pmf(m, n - k, q)

# Expected time to absorption for a given state
def compute_expected_time_for_state(k, T):
    denom = 0
    for m in range(1, n - k + 1):
        p = transition_probability(k, m)
        denom += p * T[k + m]
    return 1 + denom  # Add 1 time step + weighted future steps

# Expected time to absorption using recursion
def expected_time_to_absorption():
    T = np.zeros(n + 1)  # Expected time from each state
    T[n] = 0  # Absorbing state, time to absorption is 0

    # Use ThreadPoolExecutor to parallelize the computation
    with ThreadPoolExecutor() as executor:
        # Submit tasks for each state k from n-1 down to 0
        futures = {executor.submit(compute_expected_time_for_state, k, T): k for k in range(n - 1, -1, -1)}

        # Collect the results as they finish
        for future in futures:
            k = futures[future]
            T[k] = future.result()

    return T[0]

# Compute expected steps
expected_steps = expected_time_to_absorption()
print(f"Expected number of 6-minute steps until all type 1 cells have interacted: {expected_steps}")
print(f"Expected total time: {expected_steps * 6} minutes ≈ {expected_steps * 6 / 60:.2f} hours")

c:\Users\106682\AppData\Local\anaconda3\envs\csbdeep\lib\site-packages\scipy\stats\_discrete_distns.py:67: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


Expected number of 6-minute steps until all type 1 cells have interacted: 7.4410431985860415
Expected total time: 44.64625919151625 minutes ≈ 0.74 hours


In [35]:
from tqdm import tqdm

# Simulation parameters
n_1 = 10000 
n_2 = 10000
grid_scale = 0.31e-6
grid_cells = 5120
max_timesteps_per_simulation = 10000
num_simulations = 10 
time_per_step = 6 * 60

# Physical constants
k_b = 1.38e-23
T_kelvin = 310.15
n_vis = 0.7e-3
r_1 = 10e-6
r_2 = 5e-6
r_sum_squared = (r_1 + r_2)**2 # Radius sum squared threshold (for overlap detection)
r_1_pixels = r_1 / grid_scale 
r_2_pixels = r_2 / grid_scale
r_sum_squared_pixels = (r_1_pixels + r_2_pixels)**2

# Calculate diffusion constants (m^2/s)
D_1 = (k_b * T_kelvin) / (6 * math.pi * n_vis * r_1)
D_2 = (k_b * T_kelvin) / (6 * math.pi * n_vis * r_2)

step_size_1 = np.sqrt(2 * D_1 * time_per_step)
step_size_2 = np.sqrt(2 * D_2 * time_per_step)

step_size_1_pixels = step_size_1 / grid_scale  
step_size_2_pixels = step_size_2 / grid_scale 

print("Step size for type 1 cells (in pixels):", step_size_1_pixels)
print("Step size for type 2 cells (in pixels):", step_size_2_pixels)

def run_simulation_cell_cel_interaction():

    pos_1 = np.random.randint(0, grid_cells, size=(n_1, 2))
    pos_2 = np.random.randint(0, grid_cells, size=(n_2, 2))

    interacted = np.zeros(n_1, dtype=bool)
    print(pos_1.shape)
    # Build a KD-Tree for the positions of type 2 cells
    kdtree_type_2 = KDTree(pos_2)
    
    for t in range(max_timesteps_per_simulation):

        pos_1 += np.random.normal(0, step_size_1_pixels, size=(n_1, 2)).round().astype(int)
        pos_2 += np.random.normal(0, step_size_2_pixels, size=(n_2, 2)).round().astype(int)

        kdtree_type_2 = KDTree(pos_2)

        # Query KD-Tree for each type 1 cell to find nearby type 2 cells
        for i in range(n_1):
            # Skip cells that have already interacted
            if interacted[i]:
                continue
            # Query the KD-Tree for type 2 cells within the radius (r_1 + r_2)
            nearby_type_2_indices = kdtree_type_2.query_ball_point(pos_1[i], r_1_pixels + r_2_pixels)
            
            # If any type 2 cell is within the radius, mark type 1 cell as interacted
            if len(nearby_type_2_indices) > 0:
                interacted[i] = True

        if np.all(interacted):
            print(f"All type 1 cells have interacted by timestep {t}")
            return t, interacted
        print(f"Timestep {t} Done")
        print(f"Cells interacted {sum(interacted)}")

    return max_timesteps_per_simulation, interacted

def run_simulation_cell_cel_interaction_with_trajectory():
    # Initialize the positions of type 1 and type 2 cells
    pos_1 = np.random.randint(0, grid_cells, size=(n_1, 2))
    pos_2 = np.random.randint(0, grid_cells, size=(n_2, 2))

    interacted = np.zeros(n_1, dtype=bool)

    # Build a KD-Tree for the positions of type 2 cells
    kdtree_type_2 = KDTree(pos_2)
    
    # Lists to store the trajectories of type 1 and type 2 cells
    trajectory_1 = []  # List to store the positions of type 1 cells at each timestep
    trajectory_2 = []  # List to store the positions of type 2 cells at each timestep

    for t in range(max_timesteps_per_simulation):
        # Save current positions in the trajectory lists
        trajectory_1.append(pos_1.copy())
        trajectory_2.append(pos_2.copy())

        # Move the cells
        pos_1 += np.random.normal(0, step_size_1_pixels, size=(n_1, 2)).round().astype(int)
        pos_2 += np.random.normal(0, step_size_2_pixels, size=(n_2, 2)).round().astype(int)

        kdtree_type_2 = KDTree(pos_2)

        # Query KD-Tree for each type 1 cell to find nearby type 2 cells
        for i in range(n_1):
            # Skip cells that have already interacted
            if interacted[i]:
                continue

            # Query the KD-Tree for type 2 cells within the radius (r_1 + r_2)
            nearby_type_2_indices = kdtree_type_2.query_ball_point(pos_1[i], r_1_pixels + r_2_pixels)
            
            # If any type 2 cell is within the radius, mark type 1 cell as interacted
            if len(nearby_type_2_indices) > 0:
                interacted[i] = True

        if np.all(interacted):
            print(f"All type 1 cells have interacted by timestep {t}")
            return t, interacted, trajectory_1, trajectory_2

        print(f"Timestep {t} Done")
        print(f"Cells interacted {sum(interacted)}")
        
    # If we exit the loop without all cells interacting
    return max_timesteps_per_simulation, interacted, trajectory_1, trajectory_2

# List to store values
interaction_times = []
interacted_cells_list = []

# Run the first simulation with trajectory saving
steps, interacted_cells, traj_1, traj_2 = run_simulation_cell_cel_interaction_with_trajectory()
interaction_times.append(steps)  # Save the interaction time for the first simulation

# Now run the other 9 simulations without saving the trajectories
for _ in range(9):
    steps, interacted_cells = run_simulation_cell_cel_interaction()
    interaction_times.append(steps)
    interacted_cells_list.append(interacted_cells)

# Calculate the average interaction time
average_interaction_time = np.mean(interaction_times)

# Print the results
print(f"Interaction times for 10 simulations: {interaction_times}")
print(f"Average interaction time: {average_interaction_time:.2f} steps")


Step size for type 1 cells (in pixels): 15.589436192731686
Step size for type 2 cells (in pixels): 22.046792093511137
Timestep 0 Done
Cells interacted 9366
Timestep 1 Done
Cells interacted 9804
Timestep 2 Done
Cells interacted 9922
Timestep 3 Done
Cells interacted 9967
Timestep 4 Done
Cells interacted 9981
Timestep 5 Done
Cells interacted 9986
Timestep 6 Done
Cells interacted 9993
Timestep 7 Done
Cells interacted 9996
Timestep 8 Done
Cells interacted 9996
Timestep 9 Done
Cells interacted 9996
Timestep 10 Done
Cells interacted 9997
Timestep 11 Done
Cells interacted 9997
Timestep 12 Done
Cells interacted 9997
Timestep 13 Done
Cells interacted 9997
Timestep 14 Done
Cells interacted 9997
Timestep 15 Done
Cells interacted 9998
Timestep 16 Done
Cells interacted 9998
Timestep 17 Done
Cells interacted 9998
Timestep 18 Done
Cells interacted 9998
Timestep 19 Done
Cells interacted 9998
Timestep 20 Done
Cells interacted 9998
Timestep 21 Done
Cells interacted 9999
Timestep 22 Done
Cells interacted 

In [34]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_trajectory_to_video_cv2(trajectory_1, trajectory_2, filename="cell_trajectories.mp4", grid_size=100):
    """
    Plots the trajectory of type 1 and type 2 cells over time and saves the plot as a video using OpenCV.

    Parameters:
    - trajectory_1: List of positions for type 1 cells at each timestep (list of arrays).
    - trajectory_2: List of positions for type 2 cells at each timestep (list of arrays).
    - filename: The name of the output MP4 file.
    - grid_size: The size of the grid (default is 100x100).
    """
    
    # Set up the video writer using OpenCV
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
    fps = 1  # Frames per second
    out = cv2.VideoWriter(filename, fourcc, fps, (grid_size, grid_size))  # Output video file
    
    # Iterate over timesteps
    for t in range(len(trajectory_1)):
        # Create an empty image to plot the trajectories
        img = np.ones((grid_size, grid_size, 3), dtype=np.uint8) * 255  # White background
        
        # Plot type 1 cells (e.g., blue circles)
        for pos in trajectory_1[t]:
            cv2.circle(img, tuple(pos), 2, (255, 0, 0), -1)  # Blue color for type 1 cells

        # Plot type 2 cells (e.g., red squares)
        for pos in trajectory_2[t]:
            cv2.rectangle(img, (pos[0] - 2, pos[1] - 2), (pos[0] + 2, pos[1] + 2), (0, 0, 255), -1)  # Red color for type 2 cells
        
        # Write the current frame to the video
        out.write(img)
    
    # Release the video writer
    out.release()
    print(f"Video saved as {filename}")

plot_trajectory_to_video_cv2(traj_1, traj_2, filename="simulation_output.mp4")

Video saved as simulation_output.mp4
